In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

NameError: name 'resposta' is not defined

In [8]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio. A imersão é um curso online e gratuito, voltado para profissionais de todas as áreas, mesmo sem conhecimento prévio. O curso ensina a usar o Google Gemini na prática, com aulas sobre como criar prompts eficazes, automatizar tarefas e desenvolver agentes autônomos. Ao final do curso, os participantes recebem um certificado da Alura em parceria com o Google.

In [9]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [10]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [11]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [12]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [25]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje, links_produtos=None):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações no Google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta do google (google_search)
      para recueprar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Se forem fornecidos links de produtos, use esses links como ponto de partida para entender
      os produtos específicos do lojista e buscar notícias e informações relevantes sobre eles ou a categoria a que pertencem.
      Foque em no máximo 5 lançamentos relevantes...
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  if links_produtos:
      entrada_do_agente_buscador += f"\nLinks de produtos de referência: {', '.join(links_produtos)}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdos, especialista em criar posts para blogs de ecommerce.
        Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um texto de blog sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem
        abordados no texto para blog que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [36]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para blogs de moda.
            Você escreve posts para blogs de moda ligados a ecommerces. Utilize o tema fornecido no plano de post e os pontos mais relevantes
            fornecidos e, com base nisso, escreva um rascunho de post para blog sobre o tema indicado.
            O tamanho ideal para um texto de blog otimizado geralmente gira em torno de 1.500 a 2.000 palavras.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 tags no final.
            """,
        description="Agente redator de posts engajadores para blogs de moda"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [40]:
# %%
############################################
# --- Agente 5: Buscador de Imagens --- #
############################################
def agente_buscador_imagens(topico, rascunho_gerado):
    buscador_imagens = Agent(
        name="agente_buscador_imagens",
        model="gemini-2.0-flash",
        instruction="""
            Você é um buscador de imagens para posts de blog de moda.
            Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar imagens de alta qualidade
            e relevantes para ilustrar um post de blog sobre o tópico e rascunho fornecidos.
            Foque em encontrar imagens de sites e blogs de moda confiáveis.
            Para cada imagem encontrada, forneça a URL da imagem e a URL da página onde ela foi encontrada.
            Liste no máximo 5 imagens relevantes.
            """,
        description="Agente que busca imagens para posts de blog de moda",
        tools=[google_search] # Use a ferramenta de busca do Google
    )
    entrada_do_agente_buscador_imagens = f"Tópico: {topico}\nRascunho do post: {rascunho_gerado}"
    # Executa o agente
    imagens_encontradas = call_agent(buscador_imagens, entrada_do_agente_buscador_imagens)
    return imagens_encontradas


In [54]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts virais para blogs de moda, orientados ao SEO do Google, com foco em blogs de moda.
            Por ter um público jovem, entre 21 a 35 anos, use um tom de escrita adequado.
            Revise o rascunho de post blogs de moda, orientados ao SEO do Google abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Além disso, avalie o post com foco em SEO para o Google, considerando os seguintes aspectos:
            - **Uso de Palavras-chave:** O rascunho utiliza palavras-chave relevantes para o tópico de forma natural no título, subtítulos e ao longo do texto? Sugira palavras-chave adicionais relevantes se necessário.
            - **Estrutura do Conteúdo:** O texto está bem estruturado com títulos (H1, H2, H3) e parágrafos curtos para facilitar a leitura?
            - **Links Internos e Externos:** O rascunho inclui oportunidades para links internos (para outros produtos ou posts do blog do ecommerce) e links externos (para fontes relevantes e confiáveis)? Sugira onde links podem ser adicionados.
            - **Legibilidade:** O texto é fácil de ler para o público-alvo (linguagem simples, frases curtas)?
            - **Otimização de Imagens (Alt Text):** Se as imagens fossem incluídas, o rascunho fornece contexto suficiente para gerar bons "alt text" (descrição da imagem) com palavras-chave relevantes? (O Agente 6 cuidará da inclusão, mas o Agente 4 pode avaliar a base no rascunho).
            - **Intenção de Busca:** O conteúdo do post alinha-se com a provável intenção de busca do usuário ao procurar sobre o tópico?

            Se o rascunho estiver ótimo e otimizado para SEO, responda apenas 'O rascunho está ótimo, otimizado para SEO e pronto para publicar!'.
            Caso haja problemas de qualidade ou otimização para SEO, aponte-os de forma clara e sugira melhorias específicas para tornar o post mais forte para SEO.
            """,
        description="Agente revisor de post para blogs de moda, orientados ao SEO do Google."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [51]:
######################################################
# --- Agente 6: Formatador de Post com Imagens --- #
######################################################
def agente_formatador_imagens(rascunho_gerado, imagens_encontradas):
    formatador = Agent(
        name="agente_formatador_imagens",
        model="gemini-2.0-flash",
        instruction="""
            Você é um formatador de posts de blog, especializado em moda.
            Sua tarefa é pegar o rascunho do post e a lista de imagens encontradas e gerar o texto final do post,
            incluindo o código de incorporação das imagens nos locais apropriados dentro do texto.

            Formato de saída: Markdown. Use o seguinte formato para incorporar imagens:
            ![Descrição da Imagem](URL_DA_IMAGEM)

            Analise o texto do rascunho para identificar seções onde uma imagem seria relevante para ilustrar o conteúdo.
            Para cada imagem na lista fornecida, tente encontrar um ponto relevante no texto para inseri-la.
            Use o tópico da imagem ou o contexto do texto para gerar uma breve "Descrição da Imagem" (alt text).
            Não insira todas as imagens se elas não parecerem relevantes para o texto. Foque naquelas que melhor complementam o conteúdo.
            Posicione as imagens de forma que quebrem o texto de forma natural, talvez após um parágrafo relevante.
            Inclua as tags originais do rascunho no final do post formatado.
            """,
        description="Agente que formata o post do blog, incluindo código de incorporação de imagens em Markdown."
    )
    entrada_do_agente_formatador = f"Rascunho do Post:\n{rascunho_gerado}\n\nImagens Encontradas:\n{imagens_encontradas}"
    # Executa o agente
    post_formatado = call_agent(formatador, entrada_do_agente_formatador)
    return post_formatado

In [53]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print(" Iniciando o Sistema de Criação de Posts para blogs de moda, orientados ao SEO do Google ")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o próximo post do blog: ")

# --- Obter os Links dos Produtos do Lojista (Novo) ---
links_input = input("🔗 Opcional: Digite os links dos produtos do lojista (separados por vírgula): ")
links_do_lojista = [link.strip() for link in links_input.split(',') if link.strip()] # Processa a string de links

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre {topico}")

  # A chamada para o agente_buscador agora usará a lista links_do_lojista
  lancamentos_buscados = agente_buscador(topico, data_de_hoje, links_produtos=links_do_lojista)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("-----------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("-----------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))

  post_revisado = agente_revisor(topico, rascunho_de_post) # Renomeei para post_revisado para clareza
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_revisado))
  print("-----------------------------------------------------")

  imagens_para_post = agente_buscador_imagens(topico, rascunho_de_post)
  print("\n--- Resultado do Agente 5 (Buscador de Imagens) ---\n")
  display(to_markdown(imagens_para_post))
  print("-----------------------------------------------------")

  # Chamar o novo Agente Formatador de Post com Imagens
  post_final_formatado = agente_formatador_imagens(post_revisado, imagens_para_post)
  print("\n--- Resultado do Agente 6 (Formatador com Imagens) ---\n")
  display(to_markdown(post_final_formatado))
  print("-----------------------------------------------------")

 Iniciando o Sistema de Criação de Posts para blogs de moda, orientados ao SEO do Google 
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o próximo post do blog: camisa listrada feminina
🔗 Opcional: Digite os links dos produtos do lojista (separados por vírgula): https://www.lojatres.com.br/blusa-listrada-inv-listrado-rosa-17087072/p?property__Cor=LISTRADO%20ROSA
Maravilha! Vamos então criar o post sobre camisa listrada feminina

--- Resultado do Agente 1 (Buscador) ---



> Ok, estou procurando os lançamentos mais recentes e relevantes de camisas listradas femininas, com foco nas últimas notícias. Aqui estão as consultas que usarei:
> 
> 
> Com certeza! Aqui estão algumas das tendências e lançamentos mais recentes em camisas listradas femininas, com foco em 2025:
> 
> **Tendências Gerais:**
> 
> *   **As listras são atemporais:** As camisas listradas continuam em alta, transitando do trabalho à praia, complementando o visual de forma sofisticada, autêntica e criativa.
> *   **Versatilidade:**  Elas são perfeitas para looks formais e informais. As listras podem ser usadas em diversas peças, desde camisetas até vestidos, permitindo uma variedade infinita de combinações de estilo.
> *   **Efeito visual:** Listras verticais alongam a silhueta, enquanto as horizontais podem adicionar volume.
> *   **Mix de estampas:**  Listras combinam bem com outras padronagens como poá, xadrez ou floral.
> *   **Estilo náutico:** O estilo náutico é um clássico, principalmente nas combinações de branco com azul.
> 
> **Lançamentos e Novidades:**
> 
> *   **Verão 2025:** Camisetas listradas em canelado premium. As listras ressurgem com força total, trazendo um toque de frescor e elegância.
> *   **Inverno 2025:**  A roupa listrada (casacos, camisas, blusas e tricôs com listras horizontais) promete dominar o inverno. A camisa polo de manga longa é apontada como favorita.
> *   **Cores:** Além das clássicas listras em preto e branco, ou branco e azul, as cores ganham espaço, como o verde, vermelho, rosa, e tons neutros.
> *    **Modelos:** camisas cropped de manga longa com listras rosas e brancas, com textura frizadinha, bolsos frontais, fechamento em botões e gola blazer.
> *   **Tecidos:** Sedinha, malha de algodão, tricot modal, viscose.
> 
> **Como usar:**
> 
> *   **Casual:** Jeans, shorts ou saias, tênis ou sandálias rasteirinhas.
> *   **Moderno:** Calças de cintura alta.
> *   **Formal:** Calças de alfaiataria.
> 
> Se você busca inspiração, o Pinterest é uma ótima fonte de ideias de looks com blusas listradas.


-----------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Com base nas tendências e lançamentos mais recentes de camisas listradas femininas, podemos criar um plano de conteúdo para um blog de e-commerce abordando os seguintes pontos:
> 
> **Possíveis temas para abordar:**
> 
> 1.  **A Atemporalidade das Listras:** Explorar como as listras permanecem em alta, adaptando-se a diferentes estações e estilos.
> 2.  **Versatilidade das Camisas Listradas:** Detalhar como as listras podem ser incorporadas em looks formais e informais, e como combinar diferentes peças.
> 3.  **O Impacto Visual das Listras:** Analisar como as listras verticais e horizontais afetam a silhueta, e como usar isso a favor do seu corpo.
> 4.  **Mix de Estampas com Listras:** Oferecer dicas de como combinar listras com outras estampas, como poá, xadrez ou floral.
> 5.  **Estilo Náutico e as Listras:** Mergulhar no clássico estilo náutico, explorando as combinações de branco com azul e outras variações.
> 6.  **Tendências de Camisas Listradas para o Verão 2025:** Destacar as camisetas listradas em canelado premium, cores vibrantes e modelos cropped.
> 7.  **Camisas Listradas no Inverno 2025:** Analisar como as listras horizontais dominam o inverno, com foco em casacos, camisas, blusas e tricôs, e a popularidade da camisa polo de manga longa.
> 8.  **As Cores em Alta nas Listras:** Explorar as cores que ganham espaço nas listras, como verde, vermelho, rosa e tons neutros, além das combinações clássicas.
> 9.  **Modelos de Camisas Listradas:** Apresentar os modelos mais recentes, como camisas cropped de manga longa com listras rosas e brancas, com textura frizadinha e gola blazer.
> 10. **Tecidos Ideais para Camisas Listradas:** Indicar os tecidos mais populares para camisas listradas, como sedinha, malha de algodão, tricot modal e viscose.
> 11. **Como Usar Camisas Listradas em Looks Casuais:** Sugerir combinações com jeans, shorts ou saias, tênis ou sandálias rasteirinhas.
> 12. **Camisas Listradas em Looks Modernos:** Propor combinações com calças de cintura alta e outras peças de tendência.
> 13. **Camisas Listradas em Looks Formais:** Apresentar combinações com calças de alfaiataria e outras peças elegantes.
> 
> **Escolha do tema mais relevante:**
> 
> Entre os temas listados, o mais relevante é: **Tendências de Camisas Listradas para o Verão/Inverno 2025**. Este tema permite abordar as novidades mais recentes, as cores em alta, os modelos em destaque e os tecidos ideais para a estação.
> 
> **Plano para o texto do blog:**
> 
> 1.  **Introdução:**
>     *   Apresentar as camisas listradas como um item atemporal e versátil no guarda-roupa feminino.
>     *   Destacar a relevância das listras nas coleções de Verão e Inverno 2025.
> 2.  **Tendências de Camisas Listradas para o Verão 2025:**
>     *   Mencionar as camisetas listradas em canelado premium.
>     *   Explorar as cores vibrantes e combinações inusitadas.
>     *   Apresentar os modelos cropped e outras variações modernas.
> 3.  **Tendências de Camisas Listradas para o Inverno 2025:**
>     *   Analisar como as listras horizontais dominam o inverno.
>     *   Destacar a popularidade de casacos, camisas, blusas e tricôs com listras.
>     *   Mencionar a camisa polo de manga longa como favorita.
> 4.  **Cores e Tecidos em Destaque:**
>     *   Explorar as cores que ganham espaço nas listras, como verde, vermelho, rosa e tons neutros.
>     *   Indicar os tecidos mais populares para camisas listradas, como sedinha, malha de algodão, tricot modal e viscose.
> 5.  **Como Usar Camisas Listradas em Diferentes Ocasiões:**
>     *   Apresentar sugestões de looks casuais, modernos e formais.
>     *   Oferecer dicas de como combinar as listras com outras peças e acessórios.
> 6.  **Conclusão:**
>     *   Reforçar a versatilidade e atemporalidade das camisas listradas.
>     *   Incentivar os leitores a explorarem as diferentes tendências e estilos.
> 
> Para aprofundar a pesquisa e enriquecer o conteúdo do blog, utilizarei as seguintes pesquisas no Google:
> 
> 
> Com base nas informações coletadas e nos resultados da pesquisa, o plano de conteúdo detalhado para o tema "Tendências de Camisas Listradas para o Verão/Inverno 2025" é:
> 
> **Título:** Camisas Listradas: As Tendências Imperdíveis para o Verão e Inverno 2025
> 
> **Introdução:**
> 
> *   Apresentar as camisas listradas como um clássico atemporal e versátil, capaz de se adaptar a diferentes estilos e ocasiões.
> *   Destacar a relevância das listras nas coleções de Verão e Inverno 2025, mostrando como essa estampa se reinventa a cada temporada.
> *   Mencionar que as listras migram facilmente entre estilos, funcionando tanto em peças casuais quanto sofisticadas.
> 
> **Tendências de Camisas Listradas para o Verão 2025:**
> 
> *   Listras repaginadas e cheias de atitude, aparecendo em proporções maiores e cores vibrantes como vermelho, azul e amarelo.
> *   Estilo náutico revisitado com um toque moderno, ideal para quem busca um equilíbrio entre o casual e o chic.
> *   Cores marcantes que atualizam a estética navy, trazendo frescor e personalidade aos looks.
> *   Apresentar opções de bolsas listradas.
> *   Camisas com listras em cores intensas e contrastantes, criando um visual cheio de personalidade e frescor. Tons quentes, pastel e combinações inusitadas estão em alta, trazendo modernidade à clássica padronagem.
> 
> **Tendências de Camisas Listradas para o Inverno 2025:**
> 
> *   Dominância das listras horizontais em casacos, camisas, blusas e tricôs, indicando que essa estampa será um sucesso no frio.
> *   Camisa polo de manga longa como peça-chave da estação, sendo a favorita entre as fashionistas.
> *   Apresentar looks com a estampa clássica combinada com vermelho, criando um combo de tendências para o inverno.
> 
> **Cores e Tecidos em Destaque:**
> 
> *   Cores que ganham espaço nas listras, como verde, vermelho, rosa e tons neutros, além das combinações clássicas.
> *   Tecidos mais populares para camisas listradas, como sedinha, malha de algodão, tricot modal e viscose.
> *   Para 2025, apostar em listras mais ousadas e diversificadas, variando em largura, direção e assimetria.
> *   Novas combinações de cores, como tons pastel e neon em formatos irregulares, para looks modernos e divertidos.
> 
> **Como Usar Camisas Listradas em Diferentes Ocasiões:**
> 
> *   Looks casuais: combinações com jeans, shorts ou saias, tênis ou sandálias rasteirinhas.
> *   Looks modernos: propostas com calças de cintura alta e outras peças de tendência.
> *   Looks formais: sugestões com calças de alfaiataria e outras peças elegantes.
> *   Terceira peça: usar a camisa como terceira peça para dar um destaque no look.
> 
> **Dicas Extras:**
> 
> *   Truques de styling: usar a camisa por dentro para alongar a silhueta, dobrar as mangas para mostrar um pouco de pele.
> *   Acessórios: combinar os acessórios com os detalhes da camisa, como os metais dourados.
> 
> **Conclusão:**
> 
> *   Reafirmar a versatilidade e atemporalidade das camisas listradas, incentivando os leitores a explorarem as diferentes tendências e estilos.
> *   Incentivar os leitores a compartilhar suas próprias combinações e looks com camisas listradas.
> 
> Com este plano detalhado, o blog post terá informações relevantes, atualizadas e dicas práticas para inspirar os leitores a incorporarem as camisas listradas em seus guarda-roupas com muito estilo e personalidade.
> 


-----------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



...

-----------------------------------------------------

--- Resultado do Agente 5 (Buscador de Imagens) ---



> Okay, aqui estão algumas imagens de camisas listradas femininas que podem ser usadas para ilustrar um post de blog sobre tendências de camisas listradas em 2025.
> 
> 
> Com base nas minhas buscas, aqui estão algumas opções de imagens que podem ser relevantes para o seu post:
> 
> 1.  **Combinação com saia:** Uma mulher usando uma camisa listrada com uma saia de tecido natural, como linho, ou uma saia jeans.
>     *   URL da imagem: Não foi possível encontrar uma URL direta para a imagem.
>     *   URL da página: [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG2Q9nDdP2HsZDFDJpAdH44GwTsNMtCwdWR6eE2G-YVjgrUbmn7d1oAvnHUAS1dnV19wSlnQHXZM5oY37jkJY3EtDRmlJl1IKJoNfQZ2WrC5eD79jhbUgpuAilWofgyJpdXER8BL0OtxRufd4Zhq7mp](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG2Q9nDdP2HsZDFDJpAdH44GwTsNMtCwdWR6eE2G-YVjgrUbmn7d1oAvnHUAS1dnV19wSlnQHXZM5oY37jkJY3EtDRmlJl1IKJoNfQZ2WrC5eD79jhbUgpuAilWofgyJpdXER8BL0OtxRufd4Zhq7mp)
> 2.  **Camisa listrada com calça de alfaiataria:** Uma combinação perfeita para ambientes de trabalho, onde a camisa listrada é combinada com uma calça de alfaiataria. Para um toque mais informal, pode-se adicionar tênis.
>     *   URL da imagem: Não foi possível encontrar uma URL direta para a imagem.
>     *   URL da página: [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG2Q9nDdP2HsZDFDJpAdH44GwTsNMtCwdWR6eE2G-YVjgrUbmn7d1oAvnHUAS1dnV19wSlnQHXZM5oY37jkJY3EtDRmlJl1IKJoNfQZ2WrC5eD79jhbUgpuAilWofgyJpdXER8BL0OtxRufd4Zhq7mp](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG2Q9nDdP2HsZDFDJpAdH44GwTsNMtCwdWR6eE2G-YVjgrUbmn7d1oAvnHUAS1dnV19wSlnQHXZM5oY37jkJY3EtDRmlJl1IKJoNfQZ2WrC5eD79jhbUgpuAilWofgyJpdXER8BL0OtxRufd4Zhq7mp)
> 3.  **Camisas listradas com cores vibrantes:** Imagens de camisas listradas que incorporam cores vibrantes e até a mistura de listras com outros padrões e texturas.
>     *   URL da imagem: Não foi possível encontrar uma URL direta para a imagem.
>     *   URL da página: [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEBIEapIHiP3fQ6LIo8iXH1M-1FrzlUTMRUUv8dKOlltMc5L_KRy0sX0-D4qLmDH8CJ2daqkDMYTCS_lNq7kx-zOUJg1Br2-fgf_Cwa_VjJo50A7ZnAyTBHYdnpXi4N4dGiVv4yiD0bus-I63mLbtuiZ5U5_xJEzmXSXXQJ7qIttmNtFATLdlt3XGZWE1s_Mmkj3B9S0gYsuxkZHg==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEBIEapIHiP3fQ6LIo8iXH1M-1FrzlUTMRUUv8dKOlltMc5L_KRy0sX0-D4qLmDH8CJ2daqkDMYTCS_lNq7kx-zOUJg1Br2-fgf_Cwa_VjJo50A7ZnAyTBHYdnpXi4N4dGiVv4yiD0bus-I63mLbtuiZ5U5_xJEzmXSXXQJ7qIttmNtFATLdlt3XGZWE1s_Mmkj3B9S0gYsuxkZHg==)
> 4.  **Looks românticos com bata listrada:** Uma bata listrada com amarração e laços combinada com bermuda de alfaiataria e boné.
>     *   URL da imagem: Não foi possível encontrar uma URL direta para a imagem.
>     *   URL da página: [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEB1K8JWhngTv-OXAZxjPmPhbBpxyO9nlBcBRjOZurLr_R6aRjEibhZLsVLxNx9WhpFxTUC925ycxwLcXciD66CciPoKiyyqi3ZtP2lYiBEtseo1gxECcDFj9iDoi5k-l0xF_YGF7mesYWfMKAn6mPuQL5O3bYCmKtE4XIQpPP08_yuPNcnYCQ=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEB1K8JWhngTv-OXAZxjPmPhbBpxyO9nlBcBRjOZurLr_R6aRjEibhZLsVLxNx9WhpFxTUC925ycxwLcXciD66CciPoKiyyqi3ZtP2lYiBEtseo1gxECcDFj9iDoi5k-l0xF_YGF7mesYWfMKAn6mPuQL5O3bYCmKtE4XIQpPP08_yuPNcnYCQ=)
> 5.  **Camisa listrada azul:** Imagens de camisas listradas azuis, que podem ser combinadas de diversas formas, desde looks corporativos modernos até looks casuais e descontraídos.
>     *   URL da imagem: Não foi possível encontrar uma URL direta para a imagem.
>     *   URL da página: [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXEVN5zAcCIUrkGQqqfKyPUB37s7sty_MjJAF3pi56oU6Aqlapgr0rHEi21-Ic0D_Ecag0P_hdEzeSFnkT1pB__2ICDzuCZZhtoRKq1_EDjcI7VcgYtGwC48PrUun9N24va9N2748i2lb4Piro5zcfq5LQYzbi6Txp5jOO-tvW25irAEyYha2ydq0YyrdJnHYJZoDLwJHuNulV](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXEVN5zAcCIUrkGQqqfKyPUB37s7sty_MjJAF3pi56oU6Aqlapgr0rHEi21-Ic0D_Ecag0P_hdEzeSFnkT1pB__2ICDzuCZZhtoRKq1_EDjcI7VcgYtGwC48PrUun9N24va9N2748i2lb4Piro5zcfq5LQYzbi6Txp5jOO-tvW25irAEyYha2ydq0YyrdJnHYJZoDLwJHuNulV)
> 
> Espero que estas sugestões sejam úteis para o seu post!


-----------------------------------------------------

--- Resultado do Agente 6 (Formatador com Imagens) ---



> O rascunho está ótimo e pronto para publicar!
> 
> Looks com camisa listrada azul podem ser combinadas de diversas formas, desde looks corporativos modernos até looks casuais e descontraídos.
> 
> ![Camisa listrada azul](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFXEVN5zAcCIUrkGQqqfKyPUB37s7sty_MjJAF3pi56oU6Aqlapgr0rHEi21-Ic0D_Ecag0P_hdEzeSFnkT1pB__2ICDzuCZZhtoRKq1_EDjcI7VcgYtGwC48PrUun9N24va9N2748i2lb4Piro5zcfq5LQYzbi6Txp5jOO-tvW25irAEyYha2ydq0YyrdJnHYJZoDLwJHuNulV)
> 
> A camisa listrada também pode ser combinada com uma saia de tecido natural, como linho, ou uma saia jeans.
> 
> ![Mulher usando camisa listrada com saia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG2Q9nDdP2HsZDFDJpAdH44GwTsNMtCwdWR6eE2G-YVjgrUbmn7d1oAvnHUAS1dnV19wSlnQHXZM5oY37jkJY3EtDRmlJl1IKJoNfQZ2WrC5eD79jhbUgpuAilWofgyJpdXER8BL0OtxRufd4Zhq7mp)
> 
> A combinação perfeita para ambientes de trabalho, onde a camisa listrada é combinada com uma calça de alfaiataria. Para um toque mais informal, pode-se adicionar tênis.
> 
> ![Camisa listrada com calça de alfaiataria](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG2Q9nDdP2HsZDFDJpAdH44GwTsNMtCwdWR6eE2G-YVjgrUbmn7d1oAvnHUAS1dnV19wSlnQHXZM5oY37jkJY3EtDRmlJl1IKJoNfQZ2WrC5eD79jhbUgpuAilWofgyJpdXER8BL0OtxRufd4Zhq7mp)
> 
> Camisas listradas que incorporam cores vibrantes e até a mistura de listras com outros padrões e texturas são ótimas opções.
> 
> ![Camisas listradas com cores vibrantes](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEBIEapIHiP3fQ6LIo8iXH1M-1FrzlUTMRUUv8dKOlltMc5L_KRy0sX0-D4qLmDH8CJ2daqkDMYTCS_lNq7kx-zOUJg1Br2-fgf_Cwa_VjJo50A7ZnAyTBHYdnpXi4N4dGiVv4yiD0bus-I63mLbtuiZ5U5_xJEzmXSXXQJ7qIttmNtFATLdlt3XGZWE1s_Mmkj3B9S0gYsuxkZHg==)
> 
> Uma bata listrada com amarração e laços combinada com bermuda de alfaiataria e boné, para looks mais românticos.
> 
> ![Looks românticos com bata listrada](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEB1K8JWhngTv-OXAZxjPmPhbBpxyO9nlBcBRjOZurLr_R6aRjEibhZLsVLxNx9WhpFxTUC925ycxwLcXciD66CciPoKiyyqi3ZtP2lYiBEtseo1gxECcDFj9iDoi5k-l0xF_YGF7mesYWfMKAn6mPuQL5O3bYCmKtE4XIQpPP08_yuPNcnYCQ=)
> 


-----------------------------------------------------
